## 1 vs all training

This book assume data is ready based on train_178categories.ipynb and virtual_categories.ipynb

In [ ]:
import os
import sys
import json

import sys


import akagi
print("akagi version:", akagi.__version__)
from akagi.data_sources import RedshiftDataSource, S3DataSource
from akagi.iterator import FileFormat

import numpy as np

import pandas as pd
import glob

In [ ]:
import sys
sys.path.append("../")

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

# created by virtual_categories.ipynb
CATSTORE_PATH = ""

In [ ]:
#Data dirs {train, validation}
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [ ]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()


# Get Model from S3

In [ ]:
%%time


REGION_NAME = "ap-northeast-1"
DEFAULT_OUTPUT = "text"

DIST_DATA_DIR = os.path.normpath(BASE_DATA_DIR)

os.environ['AWS_DEFAULT_REGION'] = REGION_NAME
os.environ['AWS_DEFAULT_OUTPUT'] = DEFAULT_OUTPUT

!aws s3 sync 

In [ ]:
MODEL_DIST = './trained_model_latest'
os.path.isdir(MODEL_DIST) or os.makedirs(MODEL_DIST)
!aws s3 sync 
!aws s3 sync 

## Multiple Model Train

In [ ]:
from one_vs_all import create_multiple_model_trainer_with_weakpoint_collect, setup_filedict_by_vc

def create_multimodeltrainer():
    return create_multiple_model_trainer_with_weakpoint_collect(base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, traingen=TRAIN_DATAGEN, validgen=VALID_DATAGEN, extractor_path=CODE_EXTRACTOR)


In [ ]:
from one_vs_all import ModelEvaluator

def create_modelevaluator(valid_files_dict,  base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelEvaluator(valid_files_dict,  base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR)

# Retraining all

In [ ]:
from one_vs_all import OneVsAllModelCompiler
from category import VirtualCategories
from modelutils import load_best_model, save_model, choose_best_val_acc_path_from_base

In [ ]:
compiler = OneVsAllModelCompiler(CODE_EXTRACTOR)

In [ ]:
def codemodel_to_full_model(codef, compiler):
    bestcodef = choose_best_val_acc_path_from_base(codef)
    fullfweight = os.path.join(os.path.dirname(bestcodef), "_".join(os.path.basename(bestcodef).split("_")[1:]))
    fullf = fullfweight[:-3]
    fullmodel = compiler.codemodel_to_fullmodel(load_best_model(codef))
    save_model(fullmodel, fullf)


In [ ]:
def ensure_full_model(targetkey, compiler):
    if not os.path.exists(os.path.join(BASE_MODEL_DIR, "{}_{}.json".format(BASE_NAME, targetkey))):
        codemodel_to_full_model(os.path.join(BASE_MODEL_DIR, "code_{}_{}".format(BASE_NAME, targetkey)), compiler)

In [ ]:
multitrainer = create_multimodeltrainer()
vc = VirtualCategories.from_file(CATSTORE_PATH)
setup_filedict_by_vc(multitrainer, vc, TRAIN_DATA_DIR, VALID_DATA_DIR)
multitrainer.sample_valid_files(100, seed=123)

In [ ]:
catkeys = vc.keys()

In [ ]:
len(multitrainer.valid_files_dict['0_128'])

In [ ]:
[ensure_full_model(catkey, compiler) for catkey in catkeys[64:]]

In [ ]:
%%time

multitrainer.train_categories(catkeys[64:], eachepochs=3, retrainings=3)

In [ ]:
%%time

multitrainer.train_categories(catkeys[0:2], eachepochs=2, retrainings=3)

In [ ]:
[ensure_full_model(catkey, compiler) for catkey in catkeys[58:60]]

In [ ]:
%%time

multitrainer.train_categories(catkeys[58:60], eachepochs=3, retrainings=3)

# Convert to code model and store it to S3

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
import tqdm

In [ ]:
[best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, key) for key in tqdm.tqdm(list(catkeys[60:117]))]

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, catkeys[117])

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, catkeys[58])

In [ ]:
DIST_DIR=''

In [ ]:
os.makedirs(DIST_DIR, exist_ok=True)

In [ ]:
from codeextractor import best_code_full_path

In [ ]:
def link_codemodel_to(targetkey):
    codejson, codeweight, _, _ = best_code_full_path(BASE_MODEL_DIR, BASE_NAME, targetkey)
    os.link(codejson, os.path.join(DIST_DIR, os.path.basename(codejson)))
    os.link(codeweight, os.path.join(DIST_DIR, os.path.basename(codeweight)))
    return targetkey

In [ ]:
[link_codemodel_to(key) for key in catkeys[60:118]]

In [ ]:
link_codemodel_to(catkeys[58])

In [ ]:
S3PATH = "" + DIST_DIR

In [ ]:
# !aws s3 cp --dryrun --exclude "*" --include "code*" {DIST_DIR} {S3PATH} --recursive 
!aws s3 cp --exclude "*" --include "code*" {DIST_DIR} {S3PATH} --recursive    

In [ ]:
!aws s3 sync --exclude "*" --include "code*" {DIST_DIR} {S3PATH}  

In [ ]:
!aws s3 sync --exclude "*" --include "code*" {S3PATH} {DIST_DIR} 